In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

**Softmax**

In [ ]:
z = torch.FloatTensor([1, 2, 3])

In [ ]:
hypothesis = F.softmax(z, dim = 0)

print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [ ]:
hypothesis.sum()

tensor(1.)

**Cross Entropy Loss (Low Level)**

In [ ]:
z = torch.rand(3, 5, requires_grad = True)
hypothesis = F.softmax(z, dim = 1)

print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


In [ ]:
y = torch.randint(5, (3,)).long()

print(y)

tensor([0, 2, 1])


In [ ]:
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [ ]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim = 1).mean()

print(cost)

tensor(1.4689, grad_fn=<MeanBackward0>)


**Cross Entropy Loss with torch.nn.functional**

In [ ]:
torch.log(F.softmax(z, dim = 1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [ ]:
F.log_softmax(z, dim = 1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [ ]:
(y_one_hot * -torch.log(F.softmax(z, dim = 1))).sum(dim = 1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [ ]:
F.nll_loss(F.log_softmax(z, dim = 1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [ ]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

**Training with Low Level Cross Entropy Loss**

In [ ]:
x_train = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]

x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

In [ ]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W, b], lr = 0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    hypothesis = F.softmax(x_train.matmul(W) + b, dim = 1)
    y_one_hot = torch.zeros_like(hypothesis)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim = 1))).sum(dim = 1).mean()

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
      print("Epoch {:4d}/{}, Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000, Cost: 1.098612
Epoch  100/1000, Cost: 0.901535
Epoch  200/1000, Cost: 0.839114
Epoch  300/1000, Cost: 0.807826
Epoch  400/1000, Cost: 0.788472
Epoch  500/1000, Cost: 0.774822
Epoch  600/1000, Cost: 0.764449
Epoch  700/1000, Cost: 0.756191
Epoch  800/1000, Cost: 0.749398
Epoch  900/1000, Cost: 0.743671
Epoch 1000/1000, Cost: 0.738749


**Training with F.cross_entropy**

In [ ]:
W = torch.zeros((4, 3), requires_grad = True)
b = torch.zeros(1, requires_grad=True)

optimizer = optim.SGD([W, b], lr = 0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch {:4d}/{}, Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000, Cost: 1.098612
Epoch  100/1000, Cost: 0.761050
Epoch  200/1000, Cost: 0.689991
Epoch  300/1000, Cost: 0.643229
Epoch  400/1000, Cost: 0.604117
Epoch  500/1000, Cost: 0.568256
Epoch  600/1000, Cost: 0.533922
Epoch  700/1000, Cost: 0.500291
Epoch  800/1000, Cost: 0.466908
Epoch  900/1000, Cost: 0.433507
Epoch 1000/1000, Cost: 0.399962


**High Level Implementation with nn.Module**

In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):
        return self.linear(x)

In [ ]:
model = SoftmaxClassifierModel()

In [ ]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch {:4d}/{}, Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000, Cost: 1.845720
Epoch  100/1000, Cost: 0.647150
Epoch  200/1000, Cost: 0.568868
Epoch  300/1000, Cost: 0.515699
Epoch  400/1000, Cost: 0.471727
Epoch  500/1000, Cost: 0.432486
Epoch  600/1000, Cost: 0.395880
Epoch  700/1000, Cost: 0.360507
Epoch  800/1000, Cost: 0.325227
Epoch  900/1000, Cost: 0.289217
Epoch 1000/1000, Cost: 0.254086


In [35]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

**Cross Entropy Loss with torch.nn.functional**

In [36]:
z = torch.rand(3, 5, requires_grad = True)
hypothesis = F.softmax(z, dim = 1)

y = torch.randint(5, (3,)).long()
y_one_hot = torch.zeros_like(hypothesis)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [37]:
torch.log(F.softmax(z, dim = 1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [38]:
F.log_softmax(z, dim = 1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [39]:
(y_one_hot * -torch.log(F.softmax(z, dim = 1))).sum(dim = 1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [40]:
F.nll_loss(F.log_softmax(z, dim = 1), y.long())

tensor(1.4689, grad_fn=<NllLossBackward0>)

In [41]:
F.cross_entropy(z, y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

**Data**

In [42]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [43]:
xy = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/datafile/data-04-zoo.csv', delimiter = ',', dtype = np.float32)

In [46]:
x_train = torch.FloatTensor(xy[: , 0: -1])
y_train = torch.LongTensor(xy[: , [-1]]).squeeze()

In [47]:
print(x_train.shape)
print(len(x_train))
print(x_train[: 5])

torch.Size([101, 16])
101
tensor([[1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 4., 1., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 1., 0., 0.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 0., 0., 1.],
        [1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 4., 1., 0., 1.]])


In [48]:
print(y_train.shape)
print(len(y_train))
print(y_train[: 5])

torch.Size([101])
101
tensor([0, 0, 3, 0, 0])


In [49]:
nb_classes = 7
y_one_hot = torch.zeros((len(y_train), nb_classes))
y_one_hot = y_one_hot.scatter(1, y_train.unsqueeze(1), 1)

**Training with F.cross_entropy**

In [51]:
W = torch.zeros((16, 7), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

optimizer = optim.SGD([W, b], lr = 0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch {:4d}/{}, Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000, Cost: 1.945910
Epoch  100/1000, Cost: 0.471836
Epoch  200/1000, Cost: 0.326327
Epoch  300/1000, Cost: 0.257839
Epoch  400/1000, Cost: 0.215762
Epoch  500/1000, Cost: 0.186603
Epoch  600/1000, Cost: 0.164898
Epoch  700/1000, Cost: 0.147955
Epoch  800/1000, Cost: 0.134279
Epoch  900/1000, Cost: 0.122962
Epoch 1000/1000, Cost: 0.113422


**High level Implementation with nn.Module**

In [52]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(16, 7)
    def forward(self, x):
        return self.linear(x)

In [54]:
model = SoftmaxClassifierModel()

In [55]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000

for epoch in range(nb_epochs + 1):
    prediction = model(x_train)
    cost = F.cross_entropy(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print("Epoch {:4d}/{}. Cost: {:.6f}".format(epoch, nb_epochs, cost.item()))

Epoch    0/1000. Cost: 1.909672
Epoch  100/1000. Cost: 0.472408
Epoch  200/1000. Cost: 0.324432
Epoch  300/1000. Cost: 0.252666
Epoch  400/1000. Cost: 0.208130
Epoch  500/1000. Cost: 0.177369
Epoch  600/1000. Cost: 0.154706
Epoch  700/1000. Cost: 0.137256
Epoch  800/1000. Cost: 0.123376
Epoch  900/1000. Cost: 0.112060
Epoch 1000/1000. Cost: 0.102650
